In [2]:
import pandas as pd
import numpy as np
import csv
import xlrd
import openpyxl
from pathlib import Path
import datetime

### Read in tables from csv

In [3]:
zeta_raw = pd.read_excel('Data_Tables.xlsx', sheet_name= 'Zeta Data')
acc_bal = pd.read_excel('Data_Tables.xlsx', sheet_name= 'Account Balance')
bud = pd.read_excel('Data_Tables.xlsx', sheet_name= 'Budget')

In [4]:
zeta_raw.tail()

,date,vendor,amount,ownership,category,account
598,2021-07-15,Sp The Klassy Shop,-78.62,shared,#Clothes,Tangerine Credit
599,2021-07-15,Thank You,400.00,shared,#CreditCardPayment,Tangerine Credit
600,2021-07-15,Eft Deposit From Eq Bank,2000.00,shared,#Savings,Shared Tang Check
601,2021-07-15,Eft Deposit From Eq Bank,600.00,shared,#Savings,Shared Tang Check
602,2021-07-15,Bell Canada Ob Montreal Qc,-95.99,shared,#RentInternetUtilities,Tangerine Credit


In [5]:
acc_bal.head()

,Accounts,2019,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,TD - Shared cq,4796.85,5504.23,5018.38,4306.79,3641.07,4786.64,3366.54,NaN,NaN,NaN,NaN,NaN,NaN
1,Tang - Shared cq,10491.20,8314.76,6169.55,6740.49,6723.89,6451.65,3888.13,NaN,NaN,NaN,NaN,NaN,NaN
2,Tang - Gods cq,394.63,334.63,240.00,0.00,778.29,625.15,950.00,NaN,NaN,NaN,NaN,NaN,NaN
3,Tang - sv,41.33,64.70,50.00,62.65,86.82,111.11,50.00,NaN,NaN,NaN,NaN,NaN,NaN
4,Tang - TFSA Inv,5476.75,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
bud.head()

,Categories_Raw,Categories,Group,Flow,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,#VivCheque,Viv's Cheque,Income,In,2486.25,2350,2300,2300,2300,600,4400,NaN,NaN,NaN,NaN,NaN
1,#TimmyCheque,Timmy's Cheque,Income,In,3258,3700,3700,3700,3750,3750,4000,NaN,NaN,NaN,NaN,NaN
2,#BbPay,BB Income,Income,In,0,0,0,100,100,0,0,NaN,NaN,NaN,NaN,NaN
3,#InterestIncome,Interest Income,Income,In,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,#OtherIncome,Other Income,Income,In,10,10,10,10,10,10,10,NaN,NaN,NaN,NaN,NaN


### Turn transaction data into usable format

#### turn date into month

In [7]:

zeta_raw['monthnum'] = zeta_raw['date'].apply(lambda x: x.month) #gets month number from date column
months = {1: "Jan", 2: "Feb", 3: "Mar",4: "Apr",5: "May", 6: "Jun", 7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11:"Nov", 12:"Dec"}
zeta_raw['month'] = zeta_raw['monthnum'].apply(lambda y: months[y]) #gets month name from month number

In [8]:
#group into months and categories
zeta_grp = zeta_raw.groupby(by=['month','category']) 
zeta_sum = zeta_grp.sum().drop(columns = 'monthnum')

In [10]:
zeta_raw

,date,vendor,amount,ownership,category,account,monthnum,month
0,2021-01-01,Bookkeepers.com,-1962.44,shared,#Twork,Tangerine Credit,1,Jan
1,2021-01-01,Setup,-0.01,shared,#InterestIncome,Shared Account,1,Jan
2,2021-01-01,Setup,-0.01,shared,#TimmyCheque,Shared Account,1,Jan
3,2021-01-01,Setup,-0.01,shared,#VivCheque,Shared Account,1,Jan
4,2021-01-01,Setup,-0.01,shared,#Hobbies,Shared Account,1,Jan
...,...,...,...,...,...,...,...,...
598,2021-07-15,Sp The Klassy Shop,-78.62,shared,#Clothes,Tangerine Credit,7,Jul
599,2021-07-15,Thank You,400.00,shared,#CreditCardPayment,Tangerine Credit,7,Jul
600,2021-07-15,Eft Deposit From Eq Bank,2000.00,shared,#Savings,Shared Tang Check,7,Jul
601,2021-07-15,Eft Deposit From Eq Bank,600.00,shared,#Savings,Shared Tang Check,7,Jul


In [20]:
trans_df = zeta_raw.groupby(['month','category']).agg({'amount' : lambda act: list(act)})

In [21]:
trans_df

amount
month category                                                    
Apr   #Alcohol                                     [-46.08, -40.0]
      #BbPay                                                [20.0]
      #Car                                       [-64.23, -139.71]
      #Cellphone                                         [-168.68]
      #Clothes                            [-69.41, -110.13, -25.0]
...                                                            ...
May   #Timmyschoice                              [-162.61, -11.29]
      #Transfer      [-150.0, 150.0, -19.0, 19.0, -2900.0, 2900.0]
      #Twork                               [-6.25, -21.41, -41.16]
      #VivCheque                                [1175.18, 1170.39]
      #Vivschoice                                         [-58.76]

[164 rows x 1 columns]

In [8]:
zeta_sum.query("category == '#Savings'")

,,amount
month,category,
Apr,#Savings,-100.00
Feb,#Savings,-4400.00
Jan,#Savings,17.11
Jul,#Savings,1900.00
Jun,#Savings,1300.00
Mar,#Savings,-4317.55
May,#Savings,500.00


## Budget table

In [ ]:
#unpivot budget table so its a nice data format
bud_melt = pd.melt(bud,['Categories','Categories_Raw','Group','Flow'],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
bud_melt.dropna(inplace= True)
bud_melt.rename(columns={'value':'budget','variable':'month'},inplace = True)
bud_melt.set_index(['month','Categories_Raw'])

# Combine budget and zeta data table

In [ ]:
#merge the 2 dfs together
trans = pd.merge(zeta_sum,bud_melt,how = 'outer', left_on = ['month','category'], right_on = ['month','Categories_Raw'])
#rearrange columns
trans.drop('Categories_Raw', inplace = True, axis = 1)
trans.rename(columns = {'amount':'actual','Categories':'category','Group':'group','Flow':'flow'},inplace = True)
trans = trans[['month','category','group','flow','actual','budget']]
trans.fillna(0,inplace = True)

In [ ]:
trans

In [ ]:
#create nonth number column for indexing later
trans['monthnum'] = trans['month'].apply(lambda x: list(months.values()).index(x))+1

In [ ]:
#create the function to get the difference between budget and actual
def correct_bud(x):
    bud = x['budget']
    flow = x['flow']
    
    if flow == 'In':
        return bud
    elif flow == 'Out':
        return bud * -1
    else:
        return 0
    

trans['budget'] = trans.apply(lambda x: correct_bud(x), axis = 1)

In [ ]:
#create the function to get the difference between budget and actual
def make_diff(x):
    act = x['actual']
    bud = x['budget']
    flow = x['flow']
    
    if flow == 'In':
        diff = act - bud
    elif flow == 'Out':
        diff = bud - act
    else:
        diff = 0
    
    return diff

trans['diff'] = trans.apply(lambda x: make_diff(x), axis = 1)

In [ ]:
trans

# Pivot account table

In [ ]:
acc_bal

In [ ]:
acc = acc_bal.melt('Accounts',['2019','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
acc.rename( columns= {"Accounts": 'account','variable':'month','value':'balance'},inplace = True)
acc.dropna(inplace = True)

In [ ]:
acc

### write all the tables to csv

In [ ]:
writer = pd.ExcelWriter('Clean_Data.xlsx', engine='xlsxwriter')
acc.to_excel(writer,"Account", index = False)
trans.to_excel(writer,"Transactions",index = False)
writer.save()